## Calculo del orden y $\epsilon$ con numpy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as signal
from pytc2.sistemas_lineales import bodePlot, pzmap, GroupDelay, analyze_sys

alfa_max = 0.4
alfa_min = 48
wp = 1
ws = 3


ee =  (10**(alfa_max/10)-1)
e = ee**0.5
print("epsilon^2 = ",ee)
print("epsilon = ", e)

n = 1
print("calulamos la atenacion en omega = ws")
for n in range(1, 10, 1):
    cheby = 10*np.log10(1+ee*(np.cosh(n*np.arccosh(3))**2))
    butter = 10*np.log10(1+ee*(ws**(2*n)))
    print("order is: ", n, "alpha para butter: ", butter, "alpha para cheby: ", cheby)
    
z, p, k = signal.cheb1ap(5, alfa_max)
print(
    "\n",
    "Polos de un chebyshev de orden 5:"
)
print(p)
#print(z)
#print(k)

epsilon^2 =  0.09647819614318509
epsilon =  0.3106093948083108
calulamos la atenacion en omega = ws
order is:  1 alpha para butter:  2.714474890586518 alpha para cheby:  2.714474890586518
order is:  2 alpha para butter:  9.452092057003144 alpha para cheby:  14.606302512446167
order is:  3 alpha para butter:  18.53288084267731 alpha para cheby:  29.761586089852262
order is:  4 alpha para butter:  28.020847675655546 alpha para cheby:  45.067943217860424
order is:  5 alpha para butter:  37.55717948282837 alpha para cheby:  60.378833074960205
order is:  6 alpha para butter:  47.098927017285924 alpha para cheby:  75.689856439126
order is:  7 alpha para butter:  56.64127682071623 alpha para cheby:  91.00088373343878
order is:  8 alpha para butter:  66.18369354936635 alpha para cheby:  106.31191114344433
order is:  9 alpha para butter:  75.72611771423159 alpha para cheby:  121.62293855685556

 Polos de un chebyshev de orden 5:
[-0.11932228+1.01949544j -0.31238979+0.63008283j -0.38613502-0.j
 

## Calculo manual de $\mid T(s)\mid^2$

<img src="./tp_2_ej5_page_1.jpg" align="left" width="900"/>

<img src="./tp_2_ej5_page_2.jpg" align="left" width="900"/>

<img src="./tp_2_ej5_page_3.jpg" align="left" width="900"/>

Ahora nos quedamos con el denominador y calculamos sus raices para quedarnos con las que estan en el semiplano izquierdo

$den = -s^10 - s^8\frac{5}{2} -s^6\frac{35}{16} -s^4\frac{25}{32} -s^2\frac{25}{256} + \frac{1}{\epsilon^2 256}$

$num = \frac{1}{\epsilon^2 256}$

In [2]:
den = np.array([-1, 0, -(5/2), 0, -(35/16), 0, -(25/32), 0, -(25/256), 0, (1/(ee*256))])
#print(den)
print("\n\nsi comparamos las raices obtenidas utilizando che1ap() y las calculadas en forma manual los resultados son consistentes")
roots = np.roots(den)
print(roots)
print(p)
print("\n\n\n")
den_tf_1 = np.polymul(-1, roots[8])
den_tf_2 = np.polymul(np.array([-1, roots[2]]), np.array([-1, roots[3]]))
den_tf_3 = np.polymul(np.array([-1, roots[6]]), np.array([-1, roots[7]]))
print("den tf 1", den_tf_1)
print("den tf 2", den_tf_2)
print("den tf 3", den_tf_3)

##multiplicamos los terminos lineales de cada TF y esto debe ser igual a la raiz cuadrad del termino lineal del
## del numerador de T(s)^2 que es lo mismo que el termino lineal del numerador de T(s)
num_coef_sqrt = np.sqrt(1/(ee*256))
linear_coef_total = den_tf_1[0]*den_tf_2[2]*den_tf_3[2]
print("coeficiente lineal del numerador de T(s)", num_coef_sqrt, "\nmultiplicacion de los coeficientes lineales de los denominadores", linear_coef_total)




si comparamos las raices obtenidas utilizando che1ap() y las calculadas en forma manual los resultados son consistentes
[ 0.11932228+1.01949544j  0.11932228-1.01949544j -0.11932228+1.01949544j
 -0.11932228-1.01949544j  0.31238979+0.63008283j  0.31238979-0.63008283j
 -0.31238979+0.63008283j -0.31238979-0.63008283j -0.38613502+0.j
  0.38613502+0.j        ]
[-0.11932228+1.01949544j -0.31238979+0.63008283j -0.38613502-0.j
 -0.31238979-0.63008283j -0.11932228-1.01949544j]




den tf 1 [0.38613502+0.j]
den tf 2 [1.        +0.j 0.23864457+0.j 1.05360875+0.j]
den tf 3 [1.        +0.j 0.62477958+0.j 0.49459176+0.j]
coeficiente lineal del numerador de T(s) 0.20121735222648754 
multiplicacion de los coeficientes lineales de los denominadores (0.2012173522264874+0j)


Ahora estamos en condiciones de escribir las funciones transferencias de cada etapa:

$T_1(s) = \frac{0.386}{s+0.386}$

$T_2(s) = \frac{1.054}{s^2+s0.239+1.054}$

$T_3(s) = \frac{0.495}{s^2+s0.625+0.495}$

Recordando las ecuaciones de un filtro pasabajos RC pasivo

$\omega_0 = \frac{1}{RC}$

Adoptamos $ C_1 = 1$ y calculamos $R_1$

In [3]:
r_1 = 1/(np.abs(den_tf_1[0]))
print("R_1 = ", r_1)

R_1 =  2.5897677053768904


Recordando ecuaciones de diseño de un filtro pasivo RLC pasa bajos

$\omega_0^2 = \frac{1}{LC}$

$\frac{\omega_0}{Q}=\frac{R}{L}$

Adoptamos $C_2 = C_3 = C_1 = 1$ y calculamos $R_2, R_3, L_2 y L_3$

$L_i = \frac{1}{\omega_{0i}^2}$

$R_i = \frac{\omega_{0i}}{Q_i} L_i$

In [4]:
l_2 = 1/den_tf_2[2]
r_2 = den_tf_2[1]*l_2
print("L_2 = ", l_2, "\nR_2 = ", r_2)
l_3 = 1/den_tf_3[2]
r_3 = den_tf_3[1]*l_3
print("L_3 = ", l_3, "\nR_3 = ", r_3)

L_2 =  (0.9491189215343327+0j) 
R_2 =  (0.2265020726206374+0j)
L_3 =  (2.021869531250312+0j) 
R_3 =  (1.2632228039559747+0j)


## Simulamos el lp normalizado con respuesta de Chebyshev

En la respuesta en frecuencia se observan los 5 "toques" correspondientes a un Chebyshev de orden 5.
En $\omega = \omega_0 = 1$ pasa por 0.4dB

 <img src="./chebyshev_lp_normalizado.png" align="left" width="900"/>

 <img src="./chebyshev_lp_normalizado_ac_plot.png" align="left" width="900"/>

## Transformacion de pasa bajos a pasa altos

Usamos el nucleo de transformacion $s = \frac{1}{s}$ y lo aplicamos a $T_1(s), T_2(s) y T_3(s)$

$T_{1HP} = T_1(\frac{1}{s}) = \frac{0.386}{\frac{1}{s}+0.386} = \frac{s0.386}{s0.386 + 1} = \frac{s}{s +\frac{1}{0.386}}$

$T_{2HP} = T_2(\frac{1}{s}) =  \frac{1.054}{\left(\frac{1}{s}\right)^2+\frac{0.239}{s}+1.054} = \frac{s^2 1.054}{1+s0.239+S^2 1.054} = \frac{s^2}{s^2 + s \frac{0.239}{1.054} + \frac{1}{1.054}}$


$T_{3HP}(s) = \frac{0.495}{\left(\frac{1}{s}\right)^2+\frac{0.625}{s}+0.495} =  \frac{s^2 0.495}{1+s0.625+s^2 0.495} = \frac{s^2}{s^2+s\frac{0.625}{0.495}+ \frac{1}{0.495}} $

### Ecuaciones de filtro pasa alto pasivo de orden 1

$T_{HP} = \frac{s}{s + \frac{1}{RC}}$

$\omega_0 = \frac{1}{RC}$

### Ecuaciones de filtro pasa alto pasivo de orden 2

$T_{HP} = \frac{s^2}{s^2 + \frac{R}{L} + \frac{1}{LC}}$

$\omega_0^2 = \frac{1}{LC}$

$\frac{\omega_0}{Q} = \frac{R}{L}$

### Calculo de R1 y C1 para la etapa de primer orden

Al igual que en el filtro LP adoptamos $C_1=1$ quedando de esta forma $\omega_0 = \frac{1}{R_1} = \frac{1}{0.386} \implies R_1 = 0.386$

In [5]:
r_1_hp = r_1 = (np.abs(den_tf_1[0]))
print("R_1_HP = ", r_1_hp)

R_1_HP =  0.3861350181808949


### Calculo de R L y C para las etapas de orden dos.

Adoptamos $C_2 = C_3 = C_1 = 1$

Para la primer etapa de orden dos nos queda

$\omega_0^2 = \frac{1}{L_2} = \frac{1}{1.054} \implies L_2 = 1.054$

$\frac{\omega_0}{Q} = \frac{R}{L} = \frac{0.239}{1.054} = \frac{R_2}{L_2}  \implies R_2 = 0.239$

#### Operando de igual modo para la segunda etapa de orden 2 

$L_3 = 0.495$

$R_3 = 0.625$

In [6]:
l_2_hp = np.abs(den_tf_2[2])
r_2_hp = np.abs(den_tf_2[1])
print("L_2_HP = ", l_2_hp)
print("R_2_HP = ", r_2_hp)
l_3_hp = np.abs(den_tf_3[2])
r_3_hp = np.abs(den_tf_3[1])
print("L_3_HP = ", l_3_hp)
print("R_3_HP = ", r_3_hp)

L_2_HP =  1.0536087494530333
R_2_HP =  0.2386445654823499
L_3_HP =  0.4945917550780866
R_3_HP =  0.6247795836632473


## Simulamos el hp normalizado con respuesta de Chebyshev

 <img src="./chebyshev_hp_normalizado.png" align="left" width="900"/>

 <img src="./chebyshev_hp_normalizado_ac_plot.png" align="left" width="900"/>

### Desnormalizacion en frecuencia e impedancia

$R_i = R_i^* \times \Omega_z$

$L_i = L_i^* \times \frac{\Omega_z}{\Omega_\omega}$

$C_i = C_i^* \times \frac{1}{\Omega_z\Omega_\omega}$

In [7]:
omega_z = 2200
omega_freq = 2*np.pi*9600
c_desnormalizado = 1/(omega_z*omega_freq)
l_2_desnormalizado = l_2_hp*(omega_z/omega_freq)
l_3_desnormalizado = l_3_hp*(omega_z/omega_freq)
r_1_desnormalizado = r_1_hp*omega_z
r_2_desnormalizado = r_2_hp*omega_z
r_3_desnormalizado = r_3_hp*omega_z
print(
    "R1: ", r_1_desnormalizado,
    "\nR2: ", r_2_desnormalizado,
    "\nR3: ", r_3_desnormalizado,
    "\nL2: ", l_2_desnormalizado,
    "\nL3: ", l_3_desnormalizado,
    "\nC: ", c_desnormalizado
)


R1:  849.4970399979688 
R2:  525.0180440611698 
R3:  1374.5150840591439 
L2:  0.038428280128406794 
L3:  0.01803924893676893 
C:  7.535745411548075e-09


## Simulamos el hp normalizado con respuesta de Chebyshev

El filtro construido con los componentes desnormalizados cumple con la plantilla de diseño, para $f = 3200Hz$ la atenuacion es 60,32dB (lo esperado para un Chebyshev de orden 5) cuando $\alpha_{min} = 48dB$. Para $f=9600Hz$ se cumple con la plantilla de diseño $\alpha_{max} = 0.4dB$. En la banda de paso estan presente los 5 "toques" y la transferencia se aprozima a la unidad a medida que aumenta la frecuencia.

 <img src="./chebyshev_desnormalizado.png" align="left" width="900"/>

 <img src="./chebyshev_desnormalizado_ac_plot.png" align="left" width="900"/>
 
  <img src="./chebyshev_desnormalizado_ripple.png" align="left" width="900"/>